In [ ]:
import numpy as np
import pandas as pd

In [ ]:
Train_data = pd.read_csv("Data_Part2/traindata.csv").values
print(Train_data)

In [ ]:
words = pd.read_csv("Data_Part2/words.csv").values
#print(len(words))
print(words[0:10])
vocab_size = len(words)
print(vocab_size)
Labels = pd.read_csv("Data_Part2/trainlabel.csv").values
#print(len(Labels))
Y_train = Labels[:,0]
num_docs = len(Y_train)
print(num_docs)

In [ ]:
X_train = np.zeros((num_docs,vocab_size))
j = 0
for i in Train_data[:,0]:
    #print(i)
    X_train[i-1,Train_data[j,1]-1] = 1
    j+=1

In [ ]:
Test_data = pd.read_csv("Data_Part2/testdata.csv").values
print(Test_data)

In [ ]:
Labels_test = pd.read_csv("Data_Part2/testlabel.csv").values
#print(len(Labels))
Y_test = Labels_test[:,0]
num_docs_test = len(Y_test)
print(num_docs_test)

In [ ]:
print(X_train)
print(Y_train)

In [ ]:
# Make feature_values array
feature_values = []
for i in range(0,num_features):
    feature_values.append(list(set(X_train[:,i])))

In [ ]:
num_features = X_train.shape[1]
num_samples = X_train.shape[0]
features = []
for word in words:
    features.append(word[0])
print(features[0:10])

In [ ]:
def GINI(Y):
    #print(Y)
    nC1 = 0
    nC2 = 0
    if(np.shape(Y)==(np.shape(Y)[0],2)):
        for i in Y:
            if(i[1]==0):
                nC1+=1
            elif(i[1]==1):
                nC2+=1
        n = nC1 + nC2
        if(n==0):
            return 0
        gini = 1 - np.power((nC1/n),2) - np.power((nC2/n),2)
        return [gini,nC1,nC2]
    else:
        for i in Y:
            if(i==0):
                nC1+=1
            elif(i==1):
                nC2+=1
        n = nC1 + nC2
        gini = 1 - np.power((nC1/n),2) - np.power((nC2/n),2)
        return [gini,nC1,nC2]

In [ ]:
def split(X, Y, num_features=num_features, feature_values=feature_values, features=features):
    num_samples = X.shape[0]
    Node = []
    best_feature_idx = 0
    for i in range(0,num_features):
        L = []
        R = []
        min_gini = 1
        for j in range(0,num_samples):
            if(X[j][i]<np.mean(feature_values[i])):
                L.append([X[j],Y[j]])
            else:
                R.append([X[j],Y[j]])
        
        n1 = len(L)
        n2 = len(R)
        n = n1 + n2
        Gini_split = (n1/n)*(GINI(L)[0]) + (n2/n)*(GINI(R)[0])
        #print("\n")
        if(Gini_split<min_gini):
            min_gini = Gini_split
            best_feature_idx = i
    
    # Now we know best feature for splitting
    for j in range(0,num_samples):
        if(X[j][best_feature_idx]<np.mean(feature_values[best_feature_idx])):
            L.append([X[j],Y[j]])
        else:
            R.append([X[j],Y[j]])
    gini_L, nLC1, nLC2 = GINI(L)
    gini_R, nRC1, nRC2 = GINI(R)
    gini , nC1, nC2 = GINI(Y)
    node_class = -1
    majority_class = -1
    if(nC1>nC2):
        majority_class = 0
    else:
        majority_class = 1
    if(nC1==0):
        node_class = 1
    elif(nC2==0):
        node_class = 0
    Node_info = [gini,features[best_feature_idx],np.mean(feature_values[best_feature_idx]), node_class,majority_class]
    Node.append([Node_info,L,R])
    
    return Node

In [ ]:
Root_Node = split(X_train,Y_train)

In [ ]:
print(Root_Node)
print("\n",Root_Node[0][1])
print("\n",Root_Node[0][2])
print("\n",Root_Node[0][0])

In [ ]:
# Node_info = Node[0][0]
# L = Node[0][1]
# R = Node[0][2]
max_depth=20
DT = []
Nodes = []
Nodes.append([Root_Node[0][0][1]])
for i in range(0,max_depth):
#     Node_info = Root_Node[0][0]
#     L_Node = Root_Node[0][1]
#     R_Node = Root_Node[0][2]
    
    L_Node = split(L)
    R_Node = split(R)
    
    Nodes.append([L_Node[0][0][1]])
    Nodes.append([R_Node[0][0][1]])